In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from tqdm import tqdm as tqdm
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

In [2]:
from emu.pipeline.utils import file_ids_by_channel

In [3]:
all_files = get_data_manifest()

In [4]:
seeg_root = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/')

nev_path = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/Events.nev')
# ncs = nlx.load_ncs()

In [5]:
nev_path = os.path.join(seeg_root,'Events.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'*.ncs')))
# ncs_paths = [seeg_root+p for p in ['CSC9_0007.ncs','CSC99_0007.ncs']]

# nwb = nlx_to_nwb(nev_path,ncs_paths=ncs_paths[:5],desc='Patient 1 [Post Op Day 4]')

In [6]:
p = Participant(patient_id=1,raw_files=all_files,seeg_raw_path=seeg_root)

In [7]:
nwb = p.create_nwb(nev_path,ncs_paths[:5], desc='Patient 1 [Post Op Day 4]')

  0%|          | 0/4 [00:00<?, ?it/s]

4  blocks


100%|██████████| 4/4 [00:04<00:00,  1.07s/it]
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pd

root pynwb.file.NWBFile at 0x140269741883232
Fields:
  acquisition: {
    channel_12 <class 'pynwb.base.TimeSeries'>,
    channel_15 <class 'pynwb.base.TimeSeries'>,
    channel_18 <class 'pynwb.base.TimeSeries'>,
    channel_23 <class 'pynwb.base.TimeSeries'>,
    channel_29 <class 'pynwb.base.TimeSeries'>,
    ttl <class 'pynwb.misc.AnnotationSeries'>
  }
  file_create_date: [datetime.datetime(2020, 3, 12, 2, 45, 44, 645424, tzinfo=tzlocal())]
  identifier: 8cb63e88-69f5-480a-b894-ef5fbc9040ab
  session_description: Patient 1 [Post Op Day 4]
  session_start_time: 2020-03-06 12:37:10.586191+00:00
  timestamps_reference_time: 2020-03-06 12:37:10.586191+00:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>

In [11]:
def data_for_trial(nwb, trial):
    
    p.nwb.acquisition['channel_12']

channel_12 pynwb.base.TimeSeries at 0x140269741626648
Fields:
  comments: no comments
  conversion: 1e-06
  data: [-59.783936   -69.244385   -75.13428    ...  10.131836     4.760742
   0.45776367]
  description: no description
  rate: 1000.0
  resolution: -1.0
  starting_time: 0.0
  unit: V

In [23]:
trials = p.nwb.trials.to_dataframe()
trials['start_sample'] = (np.round(trials.start_time,3)*1000).astype(int)
trials['n_samples'] = (np.round(trials.stop_time-trials.start_time,3)*1000).astype(int)

In [24]:
for t in trials.iterrows():
    yield p.nwb.acquisition['channel_12'].data[]
trials


,start_time,stop_time,outcome,start_sample,n_samples
id,,,,,
0,208.704119,223.324801,C-C,208704,14621
1,225.637112,239.890527,C-C,225637,14253
2,240.270138,254.369968,D-C,240270,14100
3,254.536538,271.524338,C-D,254537,16988
4,268.652939,288.699086,D-C,268653,20046
5,303.035612,317.281211,C-D,303036,14246
6,323.101917,338.026257,C-D,323102,14924
7,337.351630,352.010835,D-D,337352,14659
8,352.284686,366.944583,D-D,352285,14660


In [10]:
end_trial = pdil_events.query('event == "keypress5"').session_delta.values

NameError: name 'pdil_events' is not defined

In [ ]:
trial_delta = pdil_events[pdil_events.trial>1].groupby(['block','trial']).event_delta.sum().values

In [ ]:
trial_starts = [t for d,t in zip(ttl.data,ttl.timestamps) if d.startswith('trial')]

In [ ]:
len(trial_starts)

In [ ]:
outcomes = pd.concat(p.load_game_data())

In [ ]:
outcomes = outcomes.query('block >= 3')
choices = pd.DataFrame.from_records(map(points_to_choice,outcomes.points.values),columns=['A','B'])
choices['points'] = outcomes.points.values
choices['tuple'] = [a[0].upper()+'-'+b[0].upper() for a,b in zip(choices.A.values,choices.B.values)]

In [ ]:
len(choices)

In [ ]:
nwb.add_trial_column(name='outcome',description='Choice pair for both players')

In [ ]:
for start,end,c in zip(trial_starts,trial_delta,choices.tuple.values):
    nwb.add_trial(start_time=start,stop_time=start+end,outcome=c)
#     print(start,start+end,choice)

In [ ]:
nwb.trials.to_dataframe()

In [ ]:
len(pdil_events.query('event == "keypress5"'))

In [ ]:
tasks = list(p.cache_nev())

# luigi.build(tasks,workers=5)


In [ ]:
ncs_tasks = np.random.choice(list(p.cache_ncs()),size=10,replace=False)
luigi.build(ncs_tasks,workers=5)

In [ ]:
nwb

In [ ]:
nwb.acquisition['ttl']

In [ ]:
# def ncs_to_timeseries(ncs,downsample=4):
#     rate = float(ncs['sampling_rate'])
    
# #     data_units = 
#     ch = ncs['channel_number']
#     if downsample:
#         data = ncs['data'][::downsample]
#         rate = rate/downsample
#     else:
#         data = ncs['data']
    
# #     timestamps = timestamps/10**6
#     ch_ts = TimeSeries(name='channel_{}'.format(ch),rate =rate, data=data.astype(np.float32),conversion=1.0/10**6,unit='V')
#     return ch_ts

# def create_nwb(nev_fp,ncs_paths,desc=''):
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         ncs = nlx.load_ncs(ncs_paths.pop(0))
#         nev = nlx.load_nev(nev_fp)

#     uuid = nev['header']['SessionUUID']
#     start_time = pd.to_datetime(ncs['time'][0],unit='us',utc=True).to_pydatetime()
#     start_time_sec = start_time.timestamp()
#     nwbfile = NWBFile(session_description=desc,
#                       identifier=uuid,
#                       session_start_time=start_time
#                      )
#     ts = ncs_to_timeseries(ncs)
#     nwbfile.add_acquisition(ts)
    
#     ev = pd.DataFrame.from_records(nev_as_records(nev),index='TimeStamp')
# #     print(ev.head())
#     ev['EventString'] = [str(v,'utf-8') for v in ev.EventString.values]
#     ev = ev[ev.EventString.isin(['ttl'])]
# #     ev['TimeStamp'] = ev.TimeStamp.values/10**6 - start_time.timestamp()
#     ev['time'] = pd.to_datetime(ev.index.values,unit='us')
#     ev = label_blockstart(ev)
#     ts = np.array([t.timestamp() for t in ev.time]) - start_time.timestamp()
# #     start_stop = ev[ev.EventString.isin(['Starting Recording','Stopping Recording'])]
    
#     events = AnnotationSeries(name='ttl', data=ev.label.values, timestamps=ts)
#     nwbfile.add_acquisition(events)
    
#     for p in tqdm(ncs_paths):
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             ncs = nlx.load_ncs(p)
#             nwbfile.add_acquisition(ncs_to_timeseries(ncs))
        
#     return nwbfile

In [ ]:
nwb.acquisition['channel_8']

In [ ]:
with NWBHDF5IO(os.path.join(seeg_root,'..','processed','POD4.nwb'),'w') as io:
    io.write(nwb)